### NOTE: This is being replaced by make_wavelets.ipynb 2/27/25

In [1]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

import mne.time_frequency
import mne
from ieeg.io import get_data, raw_from_layout
from ieeg.navigate import trial_ieeg, channel_outlier_marker, crop_empty_data, outliers_to_nan
from ieeg.calc.scaling import rescale
import os
from ieeg.timefreq.utils import wavelet_scaleogram, crop_pad
import numpy as np

from utils import calculate_RTs

['C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\Pythonwin']


### get wavelets for incongruent and congruent
this requires concatenating across trials for the events in the events list, note this is different from aaron's method below where each event is plotted separately instead of being concatenated.

In [ ]:
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)

output_name = "Stimulus_c25and75_fixationCrossBase_1sec"

events = ["Stimulus/c75", "Stimulus/c25"]
event_times = (-1,2)

subjects = layout.get(return_type="id", target="subject")
subjects = ["D0057", "D0059", "D0063"]

for sub in subjects:

    # Load the data
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                            extension='.edf', desc='clean', preload=False) #get line-noise filtered data
    print(filt)

    ## Crop raw data to minimize processing time
    good = crop_empty_data(filt)

    # good.drop_channels(good.info['bads'])
    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    good.drop_channels(good.info['bads'])
    # good.info['bads'] += channel_outlier_marker(good, 4, 2)
    # good.drop_channels(good.info['bads'])
    good.load_data()

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Remove intermediates from mem
    good.plot()

    ## epoching and trial outlier removal

    save_dir = os.path.join(layout.root, 'derivatives', 'spec', 'wavelet', sub)
    if not os.path.exists(save_dir):
            os.makedirs(save_dir)

    RTs, skipped = calculate_RTs(good)
    avg_RT = np.median(RTs)
    print(avg_RT)
    # make stimulus baseline EpochsTFR
    # times=[-1.5, 3.5] #this is for 0.5 sec of padding on each side. Just do 3 seconds as estimate of the full trial. Don't use avg RT because its different across subjects and makes later analysis hard.
    times = [-1.5, 0.5] #padd 0.5 secs each side, want 1 seconds before stim onset
    print("times: " + str(times))
    trials = trial_ieeg(good, "Stimulus", times, preload=True)

    # print("base trials shape is: " + str(trials._data.shape))
    outliers_to_nan(trials, outliers=10)
    base = wavelet_scaleogram(trials, n_jobs=1, decim=int(good.info['sfreq'] / 100))
    crop_pad(base, "0.5s")


    print("done with base")

    all_epochs_list = []

    for event in events:
    # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
        times_adj = [times[0] - 0.5, times[1] + 0.5]
        trials = trial_ieeg(good, event, times_adj, preload=True,
                            reject_by_annotation=False)
        all_epochs_list.append(trials)

    # Concatenate all trials
    all_trials = mne.concatenate_epochs(all_epochs_list)

    times = [None, None]
    times[0] = event_times[0] - 0.5
    times[1] = event_times[1] + 0.5

    outliers_to_nan(all_trials, outliers=10)
    spec = wavelet_scaleogram(all_trials, n_jobs=1, decim=int(good.info['sfreq'] / 100))
    crop_pad(spec, "0.5s")
    spec_a = rescale(spec, base, copy=True, mode='ratio').average(
            lambda x: np.nanmean(x, axis=0), copy=True)
    spec_a._data = np.log10(spec_a._data) * 20
    fnames = [os.path.relpath(f, layout.root) for f in good.filenames]
    spec_a.info['subject_info']['files'] = tuple(fnames)
    spec_a.info['bads'] = good.info['bads']
    filename = os.path.join(save_dir, f'{output_name}-tfr.h5')
    mne.time_frequency.write_tfrs(filename, spec_a, overwrite=True)
    spec_a.save(os.path.join(save_dir, f'{output_name}-avg.fif'), overwrite=True)

### this is for combined congruent or incongruent, it combines the events.  
it clears the outputs after the first cell cuz no memory and then runs the rest of the subjects

In [ ]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

import mne.time_frequency
import mne
from ieeg.io import get_data, raw_from_layout
from ieeg.navigate import trial_ieeg, channel_outlier_marker, crop_empty_data, outliers_to_nan
from ieeg.calc.scaling import rescale
import os
from ieeg.timefreq.utils import wavelet_scaleogram, crop_pad
import numpy as np

from misc_functions import calculate_RTs

HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)

output_name = "Stimulus_i25and75_fixationCrossBase_1sec"

events = ["Stimulus/i75", "Stimulus/i25"]
event_times = (-1,2)

subjects = layout.get(return_type="id", target="subject")
subjects = ["D0065", "D0069", "D0071"]

for sub in subjects:

    # Load the data
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                            extension='.edf', desc='clean', preload=False) #get line-noise filtered data
    print(filt)

    ## Crop raw data to minimize processing time
    good = crop_empty_data(filt)

    # good.drop_channels(good.info['bads'])
    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    good.drop_channels(good.info['bads'])
    # good.info['bads'] += channel_outlier_marker(good, 4, 2)
    # good.drop_channels(good.info['bads'])
    good.load_data()

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Remove intermediates from mem
    good.plot()

    ## epoching and trial outlier removal

    save_dir = os.path.join(layout.root, 'derivatives', 'spec', 'wavelet', sub)
    if not os.path.exists(save_dir):
            os.makedirs(save_dir)

    RTs, skipped = calculate_RTs(good)
    avg_RT = np.median(RTs)
    print(avg_RT)
    # make stimulus baseline EpochsTFR
    # times=[-1.5, 3.5] #this is for 0.5 sec of padding on each side. Just do 3 seconds as estimate of the full trial. Don't use avg RT because its different across subjects and makes later analysis hard.
    times = [-1.5, 0.5] #padd 0.5 secs each side, want 1 seconds before stim onset
    print("times: " + str(times))
    trials = trial_ieeg(good, "Stimulus", times, preload=True)

    # print("base trials shape is: " + str(trials._data.shape))
    outliers_to_nan(trials, outliers=10)
    base = wavelet_scaleogram(trials, n_jobs=1, decim=int(good.info['sfreq'] / 100))
    crop_pad(base, "0.5s")


    print("done with base")

    all_epochs_list = []

    for event in events:
    # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
        times_adj = [times[0] - 0.5, times[1] + 0.5]
        trials = trial_ieeg(good, event, times_adj, preload=True,
                            reject_by_annotation=False)
        all_epochs_list.append(trials)

    # Concatenate all trials
    all_trials = mne.concatenate_epochs(all_epochs_list)

    times = [None, None]
    times[0] = event_times[0] - 0.5
    times[1] = event_times[1] + 0.5

    outliers_to_nan(all_trials, outliers=10)
    spec = wavelet_scaleogram(all_trials, n_jobs=1, decim=int(good.info['sfreq'] / 100))
    crop_pad(spec, "0.5s")
    spec_a = rescale(spec, base, copy=True, mode='ratio').average(
            lambda x: np.nanmean(x, axis=0), copy=True)
    spec_a._data = np.log10(spec_a._data) * 20
    fnames = [os.path.relpath(f, layout.root) for f in good.filenames]
    spec_a.info['subject_info']['files'] = tuple(fnames)
    spec_a.info['bads'] = good.info['bads']
    filename = os.path.join(save_dir, f'{output_name}-tfr.h5')
    mne.time_frequency.write_tfrs(filename, spec_a, overwrite=True)
    spec_a.save(os.path.join(save_dir, f'{output_name}-avg.fif'), overwrite=True)

In [ ]:
from IPython.display import clear_output
clear_output(wait=True)

In [ ]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

import mne.time_frequency
import mne
from ieeg.io import get_data, raw_from_layout
from ieeg.navigate import trial_ieeg, channel_outlier_marker, crop_empty_data, outliers_to_nan
from ieeg.calc.scaling import rescale
import os
from ieeg.timefreq.utils import wavelet_scaleogram, crop_pad
import numpy as np

from misc_functions import calculate_RTs

HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)

output_name = "Stimulus_i25and75_fixationCrossBase_1sec"

events = ["Stimulus/i75", "Stimulus/i25"]
event_times = (-1,2)

subjects = layout.get(return_type="id", target="subject")
subjects = ["D0057", "D0059", "D0063"]

for sub in subjects:

    # Load the data
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                            extension='.edf', desc='clean', preload=False) #get line-noise filtered data
    print(filt)

    ## Crop raw data to minimize processing time
    good = crop_empty_data(filt)

    # good.drop_channels(good.info['bads'])
    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    good.drop_channels(good.info['bads'])
    # good.info['bads'] += channel_outlier_marker(good, 4, 2)
    # good.drop_channels(good.info['bads'])
    good.load_data()

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Remove intermediates from mem
    good.plot()

    ## epoching and trial outlier removal

    save_dir = os.path.join(layout.root, 'derivatives', 'spec', 'wavelet', sub)
    if not os.path.exists(save_dir):
            os.makedirs(save_dir)

    RTs, skipped = calculate_RTs(good)
    avg_RT = np.median(RTs)
    print(avg_RT)
    # make stimulus baseline EpochsTFR
    # times=[-1.5, 3.5] #this is for 0.5 sec of padding on each side. Just do 3 seconds as estimate of the full trial. Don't use avg RT because its different across subjects and makes later analysis hard.
    times = [-1.5, 0.5] #padd 0.5 secs each side, want 1 seconds before stim onset
    print("times: " + str(times))
    trials = trial_ieeg(good, "Stimulus", times, preload=True)

    # print("base trials shape is: " + str(trials._data.shape))
    outliers_to_nan(trials, outliers=10)
    base = wavelet_scaleogram(trials, n_jobs=1, decim=int(good.info['sfreq'] / 100))
    crop_pad(base, "0.5s")


    print("done with base")

    all_epochs_list = []

    for event in events:
    # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
        times_adj = [times[0] - 0.5, times[1] + 0.5]
        trials = trial_ieeg(good, event, times_adj, preload=True,
                            reject_by_annotation=False)
        all_epochs_list.append(trials)

    # Concatenate all trials
    all_trials = mne.concatenate_epochs(all_epochs_list)

    times = [None, None]
    times[0] = event_times[0] - 0.5
    times[1] = event_times[1] + 0.5

    outliers_to_nan(all_trials, outliers=10)
    spec = wavelet_scaleogram(all_trials, n_jobs=1, decim=int(good.info['sfreq'] / 100))
    crop_pad(spec, "0.5s")
    spec_a = rescale(spec, base, copy=True, mode='ratio').average(
            lambda x: np.nanmean(x, axis=0), copy=True)
    spec_a._data = np.log10(spec_a._data) * 20
    fnames = [os.path.relpath(f, layout.root) for f in good.filenames]
    spec_a.info['subject_info']['files'] = tuple(fnames)
    spec_a.info['bads'] = good.info['bads']
    filename = os.path.join(save_dir, f'{output_name}-tfr.h5')
    mne.time_frequency.write_tfrs(filename, spec_a, overwrite=True)
    spec_a.save(os.path.join(save_dir, f'{output_name}-avg.fif'), overwrite=True)

2/22 get wavelets for stimulus using new baseline of 0.2 sec before stim onset

In [ ]:
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)


layout = get_data("GlobalLocal", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")

print(subjects)

subjects = ['D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
for sub in subjects:
        # Load the data
        filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                                extension='.edf', desc='clean', preload=False) #get line-noise filtered data
        print(filt)

        ## Crop raw data to minimize processing time
        good = crop_empty_data(filt)

        # good.drop_channels(good.info['bads'])
        good.info['bads'] = channel_outlier_marker(good, 3, 2)
        good.drop_channels(good.info['bads'])
        # good.info['bads'] += channel_outlier_marker(good, 4, 2)
        # good.drop_channels(good.info['bads'])
        good.load_data()

        ch_type = filt.get_channel_types(only_data_chs=True)[0]
        good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

        # Remove intermediates from mem
        good.plot()

        ## epoching and trial outlier removal

        save_dir = os.path.join(layout.root, 'derivatives', 'spec', 'wavelet', sub)
        if not os.path.exists(save_dir):
                os.makedirs(save_dir)

        RTs, skipped = calculate_RTs(good)
        avg_RT = np.median(RTs)

        # make stimulus baseline EpochsTFR
        times = [-0.7, 0.5] #padd 0.5 secs each side, want 0.2 seconds before stim onset
        print("times: " + str(times))
        trials = trial_ieeg(good, "Stimulus", times, preload=True)

        # print("base trials shape is: " + str(trials._data.shape))
        outliers_to_nan(trials, outliers=10)
        base = wavelet_scaleogram(trials, n_jobs=1, decim=int(good.info['sfreq'] / 100))
        crop_pad(base, "0.5s")


        #now do rescale with the concatenated baseline epochs
        for event, t in [("Stimulus",(-1, 2))]:
                times = [None, None]
                times[0] = t[0] - 0.5
                times[1] = t[1] + 0.5
                trials = trial_ieeg(good, event, times, preload=True)
                outliers_to_nan(trials, outliers=10)
                spec = wavelet_scaleogram(trials, n_jobs=1, decim=int(good.info['sfreq'] / 100))
                crop_pad(spec, "0.5s")
                spec_a = rescale(spec, base, copy=True, mode='ratio').average(
                        lambda x: np.nanmean(x, axis=0), copy=True)
                spec_a._data = np.log10(spec_a._data) * 20
                fnames = [os.path.relpath(f, layout.root) for f in good.filenames]
                spec_a.info['subject_info']['files'] = tuple(fnames)
                spec_a.info['bads'] = good.info['bads']
                filename = os.path.join(save_dir, f'{event}_fixationCrossBase_0.2sec-tfr.h5')
                mne.time_frequency.write_tfrs(filename, spec_a, overwrite=True)
                spec_a.save(os.path.join(save_dir, f'{event}_fixationCrossBase_0.2sec-avg.fif'), overwrite=True)



### 7/5 try to get wavelets for all stimulus/response  
*note this is kinda old code, don't use it except for stimulus and response specifically

In [ ]:
LAB_root

In [2]:
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)


layout = get_data("GlobalLocal", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")

print(subjects)
subjects = ['D0117']

for sub in subjects:
        # Load the data
        filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                                extension='.edf', desc='clean', preload=False) #get line-noise filtered data
        print(filt)

        ## Crop raw data to minimize processing time
        good = crop_empty_data(filt)

        # good.drop_channels(good.info['bads'])
        good.info['bads'] = channel_outlier_marker(good, 3, 2)
        print(good.info['bads'])
        good.drop_channels(good.info['bads'])
        # good.info['bads'] += channel_outlier_marker(good, 4, 2)
        # good.drop_channels(good.info['bads'])
        good.load_data()

        ch_type = filt.get_channel_types(only_data_chs=True)[0]
        good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

        # Remove intermediates from mem
        good.plot()

        ## epoching and trial outlier removal

        save_dir = os.path.join(layout.root, 'derivatives', 'spec', 'wavelet', sub)
        if not os.path.exists(save_dir):
                os.makedirs(save_dir)

        RTs, skipped = calculate_RTs(good)
        avg_RT = np.median(RTs)
        
        # make stimulus baseline EpochsTFR
        times = [-1.5, 0.5] #padd 0.5 secs each side, want 1 seconds before stim onset
        print("times: " + str(times))
        trials = trial_ieeg(good, "Stimulus", times, preload=True)

        # print("base trials shape is: " + str(trials._data.shape))
        outliers_to_nan(trials, outliers=10)
        base = wavelet_scaleogram(trials, n_jobs=1, decim=int(good.info['sfreq'] / 100))
        crop_pad(base, "0.5s")


        #now do rescale with the concatenated baseline epochs
        for event, t in zip(("Stimulus", "Response"),((-1, 2), (-1, 2))):
                times = [None, None]
                times[0] = t[0] - 0.5
                times[1] = t[1] + 0.5
                trials = trial_ieeg(good, event, times, preload=True)
                outliers_to_nan(trials, outliers=10)
                spec = wavelet_scaleogram(trials, n_jobs=1, decim=int(good.info['sfreq'] / 100))
                crop_pad(spec, "0.5s")
                spec_a = rescale(spec, base, copy=True, mode='ratio').average(
                        lambda x: np.nanmean(x, axis=0), copy=True)
                spec_a._data = np.log10(spec_a._data) * 20
                fnames = [os.path.relpath(f, layout.root) for f in good.filenames]
                spec_a.info['subject_info']['files'] = tuple(fnames)
                spec_a.info['bads'] = good.info['bads']
                filename = os.path.join(save_dir, f'{event}_fullTrialBase-tfr.h5')
                mne.time_frequency.write_tfrs(filename, spec_a, overwrite=True)
                spec_a.save(os.path.join(save_dir, f'{event}_fullTrialBase-avg.fif'), overwrite=True)



['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110', 'D0117', 'D0116']
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0117\ieeg\sub-D0117_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0117\ieeg\sub-D0117_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0117\ieeg\sub-D0117_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0117\ieeg\sub-D0117_task-GlobalLocal_acq-01_run-02_desc-clean_ieeg.

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Did not find any electrodes.tsv associated with sub-D0117_task-GlobalLocal_acq-01_run-01_desc-clean.

The search_str was "C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0117\**\ieeg\sub-D0117*electrodes.tsv"
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Did not find any coordsystem.json associated with sub-D0117_task-GlobalLocal_acq-01_run-01_desc-clean.

The search_str was "C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0117\**\ieeg\sub-D0117*coordsystem.json"
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0117\ieeg\sub-D0117_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0117\ieeg\sub-D0117_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0117\ieeg\sub-D0117_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 227 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Did not find any electrodes.tsv associated with sub-D0117_task-GlobalLocal_acq-01_run-02_desc-clean.

The search_str was "C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0117\**\ieeg\sub-D0117*electrodes.tsv"
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Did not find any coordsystem.json associated with sub-D0117_task-GlobalLocal_acq-01_run-02_desc-clean.

The search_str was "C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0117\**\ieeg\sub-D0117*coordsystem.json"
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0117\ieeg\sub-D0117_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0117\ieeg\sub-D0117_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info


...

C:\Users\jz421\AppData\Local\Temp\ipykernel_10620\1772600248.py:78: RuntimeWarning: This filename (C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\spec\wavelet\D0117\Stimulus_fullTrialBase-avg.fif) does not conform to MNE naming conventions. All time-frequency object files should end with .h5 or .hdf5
  spec_a.save(os.path.join(save_dir, f'{event}_fullTrialBase-avg.fif'), overwrite=True)


Used Annotations descriptions: ['Response/c25.0/n75.0/BigLetters/SmallLetters/Taskl/TargetLetters/Responded1.0/ParticipantResponse118.0/CorrectResponse115.0/TrialCount1.0/BlockTrialCount1.0/ReactionTime2116.0/Accuracy0.0/D117', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount227.0/BlockTrialCount3.0/ReactionTime1033.0/Accuracy1.0/D117', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount229.0/BlockTrialCount5.0/ReactionTime799.0/Accuracy1.0/D117', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount233.0/BlockTrialCount9.0/ReactionTime383.0/Accuracy1.0/D117', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount247.0/BlockTrialCount23.0/ReactionTime499.0/A

[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:   12.7s
[Parallel(n_jobs=1)]: Done   4 tasks      | elapsed:   51.5s
[Parallel(n_jobs=1)]: Done   7 tasks      | elapsed:  1.5min
[Parallel(n_jobs=1)]: Done  12 tasks      | elapsed:  2.6min
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:  3.6min
[Parallel(n_jobs=1)]: Done  24 tasks      | elapsed:  5.1min
[Parallel(n_jobs=1)]: Done  31 tasks      | elapsed:  6.6min
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:  8.5min
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed: 10.5min
[Parallel(n_jobs=1)]: Done  60 tasks      | elapsed: 12.8min
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed: 15.2min
[Parallel(n_jobs=1)]: Done  84 tasks      | elapsed: 18.1min
[Parallel(n_jobs=1)]: Done  97 tasks      | elapsed: 20.8min
[Parallel(n_jobs=1)]: Done 112 tasks      | elapsed: 24.0min
[Parallel(n_jobs=1)]: Done 127 tasks      | elapsed: 27.2min
[Parallel(n_jobs=1)]: Done 144 tasks      | elapsed: 30.8min
[Parallel(n_jobs=1)]: Do

Applying baseline correction (mode: ratio)


C:\Users\jz421\AppData\Local\Temp\ipykernel_10620\1772600248.py:78: RuntimeWarning: This filename (C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\spec\wavelet\D0117\Response_fullTrialBase-avg.fif) does not conform to MNE naming conventions. All time-frequency object files should end with .h5 or .hdf5
  spec_a.save(os.path.join(save_dir, f'{event}_fullTrialBase-avg.fif'), overwrite=True)


### testing with one subject

In [ ]:
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)


layout = get_data("GlobalLocal", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")

# subjects = ["D0059", "D0063", "D0065", "D0069", "D0071"]

# for sub in subjects:

sub='D0057' #why does this not work in the for loop???
# Load the data
filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False) #get line-noise filtered data
print(filt)

## Crop raw data to minimize processing time
good = crop_empty_data(filt)

# good.drop_channels(good.info['bads'])
good.info['bads'] = channel_outlier_marker(good, 3, 2)
good.drop_channels(good.info['bads'])
# good.info['bads'] += channel_outlier_marker(good, 4, 2)
# good.drop_channels(good.info['bads'])
good.load_data()

ch_type = filt.get_channel_types(only_data_chs=True)[0]
good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

# Remove intermediates from mem
good.plot()

## epoching and trial outlier removal

save_dir = os.path.join(layout.root, 'derivatives', 'spec', 'wavelet', sub)
if not os.path.exists(save_dir):
        os.makedirs(save_dir)

RTs, skipped = calculate_RTs(good)
avg_RT = np.median(RTs)
print(avg_RT)
# make stimulus baseline EpochsTFR
# times=[-1.5, 3.5] #this is for 0.5 sec of padding on each side. Just do 3 seconds as estimate of the full trial. Don't use avg RT because its different across subjects and makes later analysis hard.
times = [-1.5, 0.5] #padd 0.5 secs each side, want 1 seconds before stim onset
print("times: " + str(times))
trials = trial_ieeg(good, "Stimulus", times, preload=True)

# print("base trials shape is: " + str(trials._data.shape))
outliers_to_nan(trials, outliers=10)
base = wavelet_scaleogram(trials, n_jobs=1, decim=int(good.info['sfreq'] / 100))
crop_pad(base, "0.5s")


print("done with base")

#now do rescale with the concatenated baseline epochs
for (event, t), filename_event in zip(zip(("Stimulus/i25", "Stimulus/c25"),((-1, 2), (-1, 2))), ("Stimulus_i25", "Stimulus_c25")):
    times = [None, None]
    times[0] = t[0] - 0.5
    times[1] = t[1] + 0.5
    trials = trial_ieeg(good, event, times, preload=True)
    print(f"trials for {event} is: " + str(trials._data.shape))
    outliers_to_nan(trials, outliers=10)
    spec = wavelet_scaleogram(trials, n_jobs=1, decim=int(good.info['sfreq'] / 100))
    crop_pad(spec, "0.5s")
    spec_a = rescale(spec, base, copy=True, mode='ratio').average(
            lambda x: np.nanmean(x, axis=0), copy=True)
    spec_a._data = np.log10(spec_a._data) * 20
    fnames = [os.path.relpath(f, layout.root) for f in good.filenames]
    spec_a.info['subject_info']['files'] = tuple(fnames)
    spec_a.info['bads'] = good.info['bads']
    filename = os.path.join(save_dir, f'{sub}_{filename_event}_1secBeforeStimOnset-tfr.h5')
    mne.time_frequency.write_tfrs(filename, spec_a, overwrite=True)
    spec_a.save(os.path.join(save_dir, f'{sub}_{filename_event}_1secBeforeStimOnset-avg.fif'), overwrite=True)
    


In [ ]:
filename = os.path.join(save_dir, f'{sub}_Stimulusi25_1secBeforeStimOnset-tfr.h5')
mne.time_frequency.write_tfrs(filename, spec_a, overwrite=True)
spec_a.save(os.path.join(save_dir, f'{sub}_Stimulusi25_1secBeforeStimOnset-avg.fif'), overwrite=True)

In [ ]:
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)


layout = get_data("GlobalLocal", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")

subjects = ["D0057", "D0059", "D0063", "D0065", "D0069", "D0071"]

for sub in subjects:

    # Load the data
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                            extension='.edf', desc='clean', preload=False) #get line-noise filtered data
    print(filt)

    ## Crop raw data to minimize processing time
    good = crop_empty_data(filt)

    # good.drop_channels(good.info['bads'])
    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    good.drop_channels(good.info['bads'])
    # good.info['bads'] += channel_outlier_marker(good, 4, 2)
    # good.drop_channels(good.info['bads'])
    good.load_data()

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Remove intermediates from mem
    good.plot()

    ## epoching and trial outlier removal

    save_dir = os.path.join(layout.root, 'derivatives', 'spec', 'wavelet', sub)
    if not os.path.exists(save_dir):
            os.makedirs(save_dir)

    RTs, skipped = calculate_RTs(good)
    avg_RT = np.median(RTs)
    print(avg_RT)
    # make stimulus baseline EpochsTFR
    # times=[-1.5, 3.5] #this is for 0.5 sec of padding on each side. Just do 3 seconds as estimate of the full trial. Don't use avg RT because its different across subjects and makes later analysis hard.
    times = [-1.5, 0.5] #padd 0.5 secs each side, want 1 seconds before stim onset
    print("times: " + str(times))
    trials = trial_ieeg(good, "Stimulus", times, preload=True)

    # print("base trials shape is: " + str(trials._data.shape))
    outliers_to_nan(trials, outliers=10)
    base = wavelet_scaleogram(trials, n_jobs=1, decim=int(good.info['sfreq'] / 100))
    crop_pad(base, "0.5s")


    print("done with base")

    #now do rescale with the concatenated baseline epochs
    for event, t in zip(("Stimulus/i25", "Stimulus/c25"),((-1, 2), (-1, 2))):
        times = [None, None]
        times[0] = t[0] - 0.5
        times[1] = t[1] + 0.5
        trials = trial_ieeg(good, event, times, preload=True)
        print(f"trials for {event} is: " + str(trials._data.shape))
        outliers_to_nan(trials, outliers=10)
        spec = wavelet_scaleogram(trials, n_jobs=1, decim=int(good.info['sfreq'] / 100))
        crop_pad(spec, "0.5s")
        spec_a = rescale(spec, base, copy=True, mode='ratio').average(
                lambda x: np.nanmean(x, axis=0), copy=True)
        spec_a._data = np.log10(spec_a._data) * 20
        fnames = [os.path.relpath(f, layout.root) for f in good.filenames]
        spec_a.info['subject_info']['files'] = tuple(fnames)
        spec_a.info['bads'] = good.info['bads']
        filename = os.path.join(save_dir, f'{sub}_{event}_1secBeforeStimOnset-tfr.h5')
        mne.time_frequency.write_tfrs(filename, spec_a, overwrite=True)
        spec_a.save(os.path.join(save_dir, f'{sub}_{event}_1secBeforeStimOnset-avg.fif'), overwrite=True)

### test it as a function

In [ ]:
import mne.time_frequency
import mne
from ieeg.io import get_data, raw_from_layout
from ieeg.navigate import trial_ieeg, channel_outlier_marker, crop_empty_data, outliers_to_nan
from ieeg.calc.scaling import rescale
import os
from ieeg.timefreq.utils import wavelet_scaleogram, crop_pad
import numpy as np
from misc_functions import calculate_RTs
from wavelet_spec_functions import get_wavelet_baseline

HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)
layout = get_data("GlobalLocal", root=LAB_root)

def get_wavelet_baseline(inst: mne.io.BaseRaw, base_times: tuple[float, float]):
    inst = inst.copy()
    inst.load_data()
    ch_type = inst.get_channel_types(only_data_chs=True)[0]
    inst.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(inst, "Stimulus", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    base = wavelet_scaleogram(trials, n_jobs=-2, decim=int(inst.info['sfreq'] / 100))
    crop_pad(base, "0.5s")
    del inst
    return base

def make_wavelets(sub, task, output_name, event=None, times=(-1, 2), base_times=(1,101), LAB_root=None):
    """
    Generate wavelets for a given subject and task using specified events.
    
    Parameters:
    - sub (str): The subject identifier.
    - task (str): The task identifier.
    - event (str, optional): The event to process. Defaults to None.
    - times (tuple, optional): A tuple indicating the start and end times for processing. Defaults to (-1, 2).
    - base_times (tuple, optional): A tuple indicating the start and end base times for processing. Defaults to (-0.5, 0).
    - LAB_root (str, optional): The root directory for the lab. Will be determined based on OS if not provided. Defaults to None.
    - output_name (str): The name for the output files.
    
    Returns:
    - spec_a: Processed data after generating wavelets.
    
    This function will process the provided events for a given subject and task. 
    Wavelets will be generated for the events and the results will be saved to output files.
    """

    # Check if LAB_root is provided. If not, determine based on OS.
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt': # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else: # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    layout = get_data(task, root=LAB_root)

    # Load the data
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub, extension='.edf', desc='clean', preload=False)

    # Crop raw data to minimize processing time
    good = crop_empty_data(filt)
    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    good.drop_channels(good.info['bads'])
    good.load_data()

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    good.plot()

    # Epoching and trial outlier removal
    save_dir = os.path.join(layout.root, 'derivatives', 'spec', 'wavelet', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    RTs, skipped = calculate_RTs(good)
    avg_RT = np.median(RTs)

    # Adjust base_times for padding
    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]

    base = get_wavelet_baseline(filt, base_times)
    # # Make stimulus baseline EpochsTFR
    # trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
    # outliers_to_nan(trials, outliers=10)
    # base = wavelet_scaleogram(trials, n_jobs=-2, decim=int(good.info['sfreq'] / 100))
    # crop_pad(base, "0.5s")

    all_trials = []

    # Adjust times for padding
    times = [times[0] - 0.5, times[1] + 0.5]
    trials = trial_ieeg(good, event, times, preload=True)
    all_trials.append(trials)
        
    print("all_trials: ", all_trials)

    # concatenate all trials
    combined_trials = mne.concatenate_epochs(all_trials)
    print("combined_trials: ", combined_trials)

    outliers_to_nan(combined_trials, outliers=10)
    spec = wavelet_scaleogram(combined_trials, n_jobs=-2, decim=int(good.info['sfreq'] / 100))
    crop_pad(spec, "0.5s")
    spec_a = rescale(spec, base, copy=True, mode='ratio').average(lambda x: np.nanmean(x, axis=0), copy=True)
    spec_a._data = np.log10(spec_a._data) * 20
    fnames = [os.path.relpath(f, layout.root) for f in good.filenames]
    spec_a.info['subject_info']['files'] = tuple(fnames)
    spec_a.info['bads'] = good.info['bads']
    filename = os.path.join(save_dir, f'{output_name}-tfr.h5')
    mne.time_frequency.write_tfrs(filename, spec_a, overwrite=True)
    spec_a.save(os.path.join(save_dir, f'{output_name}-avg.fif'), overwrite=True)
    
    return spec_a  # Optional: Return the processed data for further use.


In [ ]:
spec_a = make_wavelets(sub='D0057', task="GlobalLocal", output_name="experimentStartBase_Stimulus", event="Stimulus", times=(-1, 2))

In [ ]:
spec_a = make_wavelets(sub='D0063', task="GlobalLocal", output_name="repeat", events_list=["/r"], times=(-1, 2))